In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import sys
import os
from tqdm import tqdm_notebook as tqdm

#sys.path.append("..")

import pickle
import matplotlib.pyplot as plt
import copy
from datetime import datetime
import time
from io import StringIO

from tqdm import tqdm
import argparse
#import sys
# sys.path.append('/content/drive/MyDrive')

# # Step 3: Now you can import the utils module
# import utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
#sys.path.append("..")
#import utils
import pickle
import copy
import csv
from datetime import datetime
import time
from io import StringIO
from tqdm import tqdm as tqdm
import pandas as pd

In [ ]:
D2C_MIN = 2000 #meters




LAT_MIN = 23.565
LAT_MAX = 30.10224
LON_MIN = -96.554
LON_MAX = -82.784

In [ ]:
dataset_path = "/content/drive/MyDrive/15 month new"

# List of CSV files
# l_csv_filename = [
#     "train.csv"
#     #,"valid.csv"
#     #"Est-aruba_5x5deg_2019240_2019365_.csv"
# ]



In [ ]:
pkl_filename = "ct_2019\\ct_2017010203_10_20_simulated.pkl"
pkl_filename_train = "ct_2019\\ct_2017010203_10_20_train.pkl"
pkl_filename_valid = "ct_2019\\ct_2017010203_10_20_valid.pkl"
pkl_filename_test = "ct_2019\\ct_2017010203_10_20_test.pkl"

In [ ]:
LAT_RANGE = LAT_MAX - LAT_MIN
LON_RANGE = LON_MAX - LON_MIN
SOG_MAX = 30.0

In [ ]:
EPOCH = datetime(1970, 1, 1)
LAT, LON, SOG, COG, HEADING, NAV_STT, TIMESTAMP, MMSI, SHIPTYPE = list(range(9))

In [ ]:
l_l_msg = [] # list of AIS messages, each row is a message (list of AIS attributes)
n_error = 0
#test_csv_filename=["valid.csv"]
import pandas as pd
for csv_filename in os.listdir(dataset_path):
    data_path = os.path.join(dataset_path,csv_filename)

    df=pd.read_csv(data_path)
    print(f"reading: {csv_filename}")
    sog = df['SOG']  # Column index 4 is 'SOG'
    status = df['Status']  # Column index 11 is 'Status'

    # Create a new column for the filled status using numpy's select for vectorized conditions
    conditions = [
        status.isnull() & (sog == 0.0),
        status.isnull() & (0.1 <= sog) & (sog <= 5.0),
        status.isnull() & (0.1 < sog) & (sog <= 8.0),
        status.isnull() & (5.1 <= sog) & (sog <= 10.0),
        status.isnull() & (10.1 <= sog) & (sog <= 15.0),
        status.isnull() & (15.1 <= sog) & (sog <= 30.0),
        status.isnull() & (sog > 30.0)
    ]

    choices = [
        6,  # Moored (Status 6)
        3,  # Not Under Command (Status 3)
        8,  # Engaged in Fishing (Status 8)
        4,  # Restricted Maneuverability (Status 4)
        1,  # Underway Using Engine (Status 1)
        1,  # Underway Using Engine (Status 1)
        11  # High-Speed Craft (Status 11)
    ]

    # Default status for other cases
    #default_status = 1

    # Apply the conditions and choices, fill in the status where it's NaN
    df['Status_new'] = np.select(conditions, choices, status)
    df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])
    df['unix'] = df['BaseDateTime'].astype('int64') // 10**9
    l_msg = np.column_stack([
    df['LAT'].astype(float),           # Latitude as float
    df['LON'].astype(float),           # Longitude as float
    df['SOG'].astype(float),           # Speed over ground as float
    df['COG'].astype(float),           # Course over ground as float
    df['Heading'].astype(float),       # Heading as float
    df['Status_new'].astype(int),      # Status as integer
    df['unix'].astype(int),            # Unix timestamp as integer
    df['MMSI'].astype(int),            # MMSI as integer
    df['VesselType'].astype(int)       # Vessel type as integer
]).tolist()
    l_l_msg.extend(l_msg)



reading: AIS_172998267124367251_2326-1729982682125.csv
reading: AIS_172998267124367251_2580-1729988258904.csv
reading: AIS_172998267124367251_3002-1729996853879.csv
reading: AIS_172998267124367251_2818-1729993194492.csv
reading: AIS_172998267124367251_3165-1729997206940.csv
reading: AIS_172998267124367251_3609-1730002225221.csv
reading: AIS_172998378920267252_3751-1729983792111.csv
reading: AIS_172998378920267252_4013-1729989594335.csv
reading: AIS_172998378920267252_7924-1729995325742.csv
reading: AIS_172998378920267252_8319-1730000707261.csv
reading: AIS_172998378920267252_8629-1730003940025.csv
reading: AIS_172998441826067253_10116-1729989566328.csv
reading: AIS_172998441826067253_10264-1729994722223.csv
reading: AIS_172998441826067253_10559-1729999529421.csv
reading: AIS_172998441826067253_11053-1730005832397.csv
reading: AIS_172998441826067253_11316-1730009814786.csv
reading: AIS_172998441826067253_11882-1730010195940.csv
reading: AIS_172998441826067253_9482-1729984423790.csv
read

In [ ]:
print(len(l_l_msg))

55944244


In [ ]:
l_l_msg[0]

[29.30631, -94.49835, 0.1, 104.8, 334.0, 1.0, 1578009717.0, 353775000.0, 70.0]

In [ ]:
data_path = "/content/drive/MyDrive/train.csv"
df_test=pd.read_csv(data_path)
sog = df_test['SOG']  # Column index 4 is 'SOG'
status = df_test['Status']  # Column index 11 is 'Status'

# Create a new column for the filled status using numpy's select for vectorized conditions
conditions = [
    status.isnull() & (sog == 0.0),
    status.isnull() & (0.1 <= sog) & (sog <= 5.0),
    status.isnull() & (0.1 < sog) & (sog <= 8.0),
    status.isnull() & (5.1 <= sog) & (sog <= 10.0),
    status.isnull() & (10.1 <= sog) & (sog <= 15.0),
    status.isnull() & (15.1 <= sog) & (sog <= 30.0),
    status.isnull() & (sog > 30.0)
]

choices = [
    6,  # Moored (Status 6)
    3,  # Not Under Command (Status 3)
    8,  # Engaged in Fishing (Status 8)
    4,  # Restricted Maneuverability (Status 4)
    1,  # Underway Using Engine (Status 1)
    1,  # Underway Using Engine (Status 1)
    11  # High-Speed Craft (Status 11)
]

# Default status for other cases
#default_status = 1

# Apply the conditions and choices, fill in the status where it's NaN
df_test['Status_new'] = np.select(conditions, choices, status)
df_test['BaseDateTime'] = pd.to_datetime(df_test['BaseDateTime'])
df_test['unix'] = df_test['BaseDateTime'].astype('int64') // 10**9
df=df_test
test_l_msg = np.column_stack([
    df['LAT'].astype(float),           # Latitude as float
    df['LON'].astype(float),           # Longitude as float
    df['SOG'].astype(float),           # Speed over ground as float
    df['COG'].astype(float),           # Course over ground as float
    df['Heading'].astype(float),       # Heading as float
    df['Status_new'].astype(int),      # Status as integer
    df['unix'].astype(int),            # Unix timestamp as integer
    df['MMSI'].astype(int),            # MMSI as integer
    df['VesselType'].astype(int)       # Vessel type as integer
]).tolist()
#LAT, LON, SOG, COG, HEADING, NAV_STT, TIMESTAMP, MMSI, SHIPTYPE = list(range(9))
# Display the result
#print(list_of_lists)


In [ ]:
print(len(test_l_msg))
test_l_msg[0]

15696148


[29.37895, -85.61354, 7.7, 283.5, 511.0, 0.0, 1561616697.0, 367298000.0, 90.0]

In [ ]:
m_msg = np.array(l_l_msg)
del l_l_msg
print("Total number of AIS messages: ",m_msg.shape[0])

print("Lat min: ",np.min(m_msg[:,LAT]), "Lat max: ",np.max(m_msg[:,LAT]))
print("Lon min: ",np.min(m_msg[:,LON]), "Lon max: ",np.max(m_msg[:,LON]))
print("Ts min: ",np.min(m_msg[:,TIMESTAMP]), "Ts max: ",np.max(m_msg[:,TIMESTAMP]))

# Convert to suitable timestamp format

print("Time min: ",datetime.utcfromtimestamp(np.min(m_msg[:,TIMESTAMP])).strftime('%Y-%m-%d %H:%M:%SZ'))
print("Time max: ",datetime.utcfromtimestamp(np.max(m_msg[:,TIMESTAMP])).strftime('%Y-%m-%d %H:%M:%SZ'))



Total number of AIS messages:  55944244
Lat min:  23.49571 Lat max:  30.23999
Lon min:  -97.00537 Lon max:  -82.68705
Ts min:  1578009600.0 Ts max:  1617321597.0
Time min:  2020-01-03 00:00:00Z
Time max:  2021-04-01 23:59:57Z


In [ ]:
m_msgv = np.array(test_l_msg)
del test_l_msg
print("Total number of AIS messages: ",m_msg.shape[0])

print("Lat min: ",np.min(m_msgv[:,LAT]), "Lat max: ",np.max(m_msgv[:,LAT]))
print("Lon min: ",np.min(m_msgv[:,LON]), "Lon max: ",np.max(m_msgv[:,LON]))
print("Ts min: ",np.min(m_msgv[:,TIMESTAMP]), "Ts max: ",np.max(m_msgv[:,TIMESTAMP]))

# Convert to suitable timestamp format

print("Time min: ",datetime.utcfromtimestamp(np.min(m_msgv[:,TIMESTAMP])).strftime('%Y-%m-%d %H:%M:%SZ'))
print("Time max: ",datetime.utcfromtimestamp(np.max(m_msgv[:,TIMESTAMP])).strftime('%Y-%m-%d %H:%M:%SZ'))



Total number of AIS messages:  55944244
Lat min:  23.565 Lat max:  30.10224
Lon min:  -96.55398 Lon max:  -82.784
Ts min:  1557100800.0 Ts max:  1611100800.0
Time min:  2019-05-06 00:00:00Z
Time max:  2021-01-20 00:00:00Z


In [ ]:
m_msg = m_msg[m_msg[:,LAT]>=LAT_MIN]
m_msg = m_msg[m_msg[:,LAT]<=LAT_MAX]
m_msg = m_msg[m_msg[:,LON]>=LON_MIN]
m_msg = m_msg[m_msg[:,LON]<=LON_MAX]
# SOG
m_msg = m_msg[m_msg[:,SOG]>=0]
m_msg = m_msg[m_msg[:,SOG]<=SOG_MAX]
# COG
m_msg = m_msg[m_msg[:,SOG]>=0]
m_msg = m_msg[m_msg[:,COG]<=360]
# D2C
#m_msg = m_msg[m_msg[:,D2C]>=D2C_MIN]

# TIME
m_msg = m_msg[m_msg[:,TIMESTAMP]>=0]

In [ ]:
m_msgv = m_msgv[m_msgv[:,LAT]>=LAT_MIN]
m_msgv = m_msgv[m_msgv[:,LAT]<=LAT_MAX]
m_msgv = m_msgv[m_msgv[:,LON]>=LON_MIN]
m_msgv = m_msgv[m_msgv[:,LON]<=LON_MAX]
# SOG
m_msgv = m_msgv[m_msgv[:,SOG]>=0]
m_msgv = m_msgv[m_msgv[:,SOG]<=SOG_MAX]
# COG
m_msgv = m_msgv[m_msgv[:,SOG]>=0]
m_msgv = m_msgv[m_msgv[:,COG]<=360]
# D2C
#m_msgv = m_msgv[m_msgv[:,D2C]>=D2C_MIN]

# TIME
m_msgv = m_msgv[m_msgv[:,TIMESTAMP]>=0]

In [ ]:
m_msg[0],m_msgv[0]

(array([ 2.93063100e+01, -9.44983500e+01,  1.00000000e-01,  1.04800000e+02,
         3.34000000e+02,  1.00000000e+00,  1.57800972e+09,  3.53775000e+08,
         7.00000000e+01]),
 array([ 2.9378950e+01, -8.5613540e+01,  7.7000000e+00,  2.8350000e+02,
         5.1100000e+02,  0.0000000e+00,  1.5616167e+09,  3.6729800e+08,
         9.0000000e+01]))

In [ ]:
print("Total msgs: ",len(m_msg))
m_msg_train=m_msg
del m_msg
m_msg_valid=m_msgv
#del m_msgv
m_msg_test=m_msgv
del m_msgv
print("Number of msgs in the training set: ",len(m_msg_train))
print("Number of msgs in the validation set: ",len(m_msg_valid))
print("Number of msgs in the test set: ",len(m_msg_test))


## MERGING INTO DICT
#======================================
# Creating AIS tracks from the list of AIS messages.
# Each AIS track is formatted by a dictionary.


Total msgs:  54568096


NameError: name 'm_msgv' is not defined

In [ ]:
print("Convert to dicts of vessel's tracks...")

# Training set
Vs_train = dict()
for v_msg in tqdm(m_msg_train):
    mmsi = int(v_msg[MMSI])
    if not (mmsi in list(Vs_train.keys())):
        Vs_train[mmsi] = np.empty((0,9))
    Vs_train[mmsi] = np.concatenate((Vs_train[mmsi], np.expand_dims(v_msg[:9],0)), axis = 0)

del m_msg_train
# Validation set
Vs_valid = dict()
for v_msg in tqdm(m_msg_valid):
    mmsi = int(v_msg[MMSI])
    if not (mmsi in list(Vs_valid.keys())):
        Vs_valid[mmsi] = np.empty((0,9))
    Vs_valid[mmsi] = np.concatenate((Vs_valid[mmsi], np.expand_dims(v_msg[:9],0)), axis = 0)

#del m_msg_valid
# Test set
Vs_test = dict()
for v_msg in tqdm(m_msg_valid):
    mmsi = int(v_msg[MMSI])
    if not (mmsi in list(Vs_test.keys())):
        Vs_test[mmsi] = np.empty((0,9))
    Vs_test[mmsi] = np.concatenate((Vs_test[mmsi], np.expand_dims(v_msg[:9],0)), axis = 0)
del m_msg_valid


Convert to dicts of vessel's tracks...


 68%|██████▊   | 36979674/54568096 [2:32:14<1:54:22, 2563.13it/s]

In [ ]:
for key in Vs_train.keys():
  print(len(Vs_train[key]))

In [ ]:
for filename, filedict in zip([pkl_filename_train,pkl_filename_valid,pkl_filename_test],
                              [Vs_train,Vs_valid,Vs_test]
                             ):
    print("Writing to ", os.path.join(dataset_path,filename),"...")
    with open(os.path.join(dataset_path,filename),"wb") as f:
        pickle.dump(filedict,f)
    print("Total number of tracks: ", len(filedict))

In [ ]:
LAT_RANGE = LAT_MAX - LAT_MIN
LON_RANGE = LON_MAX - LON_MIN
SPEED_MAX = 30.0  # knots
DURATION_MAX = 24 #h

In [ ]:
FIG_W = 960
FIG_H = int(960*LAT_RANGE/LON_RANGE) #533 #768

import os
import pickle

dataset_dir = '/content/drive/MyDrive/data2'

# Initialize a list to store the loaded data
dict_list = []

# Loop through each file in the directory
for filename in os.listdir(dataset_dir):
    if filename.endswith('.pkl'):
        # Load each .pkl file and append to dict_list
        with open(os.path.join(dataset_dir, filename), "rb") as f:
            temp = pickle.load(f)
            dict_list.append(temp)

In [ ]:
len(dict_list)

In [ ]:
pio=0
for Vi in dict_list:
  for mmsi in list(Vi.keys()):
      pio+=1
print(pio)

In [ ]:
print(" Remove erroneous timestamps and erroneous speeds...")
Vs = dict()
for Vi in dict_list:
    #print(filename)

    for mmsi in list(Vi.keys()):
        # Boundary
        lat_idx = np.logical_or((Vi[mmsi][:,LAT] > LAT_MAX),
                                (Vi[mmsi][:,LAT] < LAT_MIN))
        Vi[mmsi] = Vi[mmsi][np.logical_not(lat_idx)]
        lon_idx = np.logical_or((Vi[mmsi][:,LON] > LON_MAX),
                                (Vi[mmsi][:,LON] < LON_MIN))
        Vi[mmsi] = Vi[mmsi][np.logical_not(lon_idx)]
#         # Abnormal timestamps
#         abnormal_timestamp_idx = np.logical_or((Vi[mmsi][:,TIMESTAMP] > t_max),
#                                                (Vi[mmsi][:,TIMESTAMP] < t_min))
#         Vi[mmsi] = Vi[mmsi][np.logical_not(abnormal_timestamp_idx)]
        # Abnormal speeds
        abnormal_speed_idx = Vi[mmsi][:,SOG] > SPEED_MAX
        Vi[mmsi] = Vi[mmsi][np.logical_not(abnormal_speed_idx)]
        # Deleting empty keys
        if len(Vi[mmsi]) == 0:
            del Vi[mmsi]
            continue
        if mmsi not in list(Vs.keys()):
            Vs[mmsi] = Vi[mmsi]
            del Vi[mmsi]
        else:
            Vs[mmsi] = np.concatenate((Vs[mmsi],Vi[mmsi]),axis = 0)
            del Vi[mmsi]
#del dict_list, Vi, abnormal_speed_idx

In [ ]:
del dict_list, Vi, abnormal_speed_idx

In [ ]:
print(len(Vs))

In [ ]:
Vs

In [ ]:
## STEP 2: VOYAGES SPLITTING
#======================================
# Cutting discontiguous voyages into contiguous ones
print("Cutting discontiguous voyages into contiguous ones...")
count = 0
voyages = dict()
INTERVAL_MAX = 2*3600 # 2h
for mmsi in list(Vs.keys()):
    v = Vs[mmsi]
    # Intervals between successive messages in a track
    intervals = v[1:,TIMESTAMP] - v[:-1,TIMESTAMP]
    idx = np.where(intervals > INTERVAL_MAX)[0]
    if len(idx) == 0:
        voyages[count] = v
        count += 1
    else:
        tmp = np.split(v,idx+1)
        for t in tmp:
            voyages[count] = t
            count += 1



In [ ]:
print(len(voyages))

In [ ]:
print("Removing AIS track whose length is smaller than 20 or those last less than 4h...")

for k in list(voyages.keys()):
    duration = voyages[k][-1,TIMESTAMP] - voyages[k][0,TIMESTAMP]
    if (len(voyages[k]) < 20) or (duration < 4*3600):
        voyages.pop(k, None)

In [ ]:
print(len(voyages))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
from scipy import interpolate
import scipy.ndimage as ndimage
from math import radians, cos, sin, asin, sqrt
import sys
# sys.path.append('..')
# sys.path.append('Data')
#import shapefile
import time
from pyproj import Geod
geod = Geod(ellps='WGS84')
#import dataset

AVG_EARTH_RADIUS = 6378.137  # in km
SPEED_MAX = 30 # knot
FIG_DPI = 150

#LAT, LON, SOG, COG, HEADING, ROT, NAV_STT, TIMESTAMP, MMSI = list(range(9))

def trackOutlier(A):
    """
    Koyak algorithm to perform outlier identification
    Our approach to outlier detection is to begin by evaluating the expression
    “observation r is anomalous with respect to observation s ” with respect to
    every pair of measurements in a track. We address anomaly criteria below;
    assume for now that a criterion has been adopted and that the anomaly
    relationship is symmetric. More precisely, let a(r,s) = 1 if r and s are
    anomalous and a(r,s) = 0 otherwise; symmetry implies that a(r,s) = a(s,r).
    If a(r,s) = 1 either one or both of observations are potential outliers,
    but which of the two should be treated as such cannot be resolved using
    this information alone.
    Let A denote the matrix of anomaly indicators a(r, s) and let b denote
    the vector of its row sums. Suppose that observation r is an outlier and
    that is the only one present in the track. Because we expect it to be
    anomalous with respect to many if not all of the other observations b(r)
    should be large, while b(s) = 1 for all s ≠ r . Similarly, if there are
    multiple outliers the values of b(r) should be large for those observations
    and small for the non-outliers.
    Source: "Predicting vessel trajectories from AIS data using R", Brian L
    Young, 2017
    INPUT:
        A       : (nxn) symmatic matrix of anomaly indicators
    OUTPUT:
        o       : n-vector outlier indicators

    # FOR TEST
    A = np.zeros((5,5))
    idx = np.array([[0,2],[1,2],[1,3],[0,3],[2,4],[3,4]])
    A[idx[:,0], idx[:,1]] = 1
    A[idx[:,1], idx[:,0]] = 1    sampling_track = np.empty((0, 9))
    for t in range(int(v[0,TIMESTAMP]), int(v[-1,TIMESTAMP]), 300): # 5 min
        tmp = utils.interpolate(t,v)
        if tmp is not None:
            sampling_track = np.vstack([sampling_track, tmp])
        else:
            sampling_track = None
            break
    """
    assert (A.transpose() == A).all(), "A must be a symatric matrix"
    assert ((A==0) | (A==1)).all(), "A must be a binary matrix"
    # Initialization
    n = A.shape[0]
    b = np.sum(A, axis = 1)
    o = np.zeros(n)
    while(np.max(b) > 0):
        r = np.argmax(b)
        o[r] = 1
        b[r] = 0
        for j in range(n):
            if (o[j] == 0):
                b[j] -= A[r,j]
    return o.astype(bool)

#===============================================================================
#===============================================================================
def detectOutlier(track, speed_max = SPEED_MAX):
    """
    removeOutlier() removes anomalus AIS messages from AIS track.
    An AIS message is considered as beging anomalous if the speed is
    infeasible (> speed_max). There are two types of anomalous messages:
        - The reported speed is infeasible
        - The calculated speed (distance/time) is infeasible

    INPUT:
        track       : a (nxd) matrix. Each row is an AIS message. The structure
                      must follow: [Timestamp, Lat, Lon, Speed]
        speed_max   : knot
    OUTPUT:
        o           : n-vector outlier indicators
    """
    # Remove anomalous reported speed
    o_report = track[:,3] > speed_max # Speed in track is in knot
    if o_report.all():
        return o_report, None
    track = track[np.invert(o_report)]
    # Calculate speed base on (lon, lat) and time

    N = track.shape[0]
    # Anomoly indicator matrix
    A = np.zeros(shape = (N,N))

    # Anomalous calculated-speed
    for i in range(1,5):
        # the ith diagonal
        _, _, d = geod.inv(track[:N-i,2],track[:N-i,1],
                           track[i:,2],track[i:,1])
        delta_t = track[i:,0] - track[:N-i,0].astype(np.float64)
        cond = np.logical_and(delta_t > 2,d/(delta_t+1e-6) > (speed_max*0.514444))
        abnormal_idx = np.nonzero(cond)[0]
        A[abnormal_idx, abnormal_idx + i] = 1
        A[abnormal_idx + i, abnormal_idx] = 1

    o_calcul = trackOutlier(A)

    return o_report, o_calcul

In [ ]:
def createShapefile(shp_fname, Vs):
    """
    Creating AIS shape files
    INPUT:
        shp_fname    : name of the shapefile
        Vs          : AIS data, each element of the dictionary is an AIS track
                      whose structure is:
                      [Timestamp, MMSI, Lat, Lon, SOG, COG, Heading, ROT, NAV_STT]
    """
    shp = shapefile.Writer(shapefile.POINT)
    shp.field('MMSI', 'N', 10)
    shp.field('TIMESTAMP', 'N', 12)
    shp.field('DATETIME', 'C', 20)
    shp.field('LAT','N',10,5)
    shp.field('LON','N',10,5)
    shp.field('SOG','N', 10,5)
    shp.field('COG', 'N', 10,5)
    shp.field('HEADING', 'N', 10,5)
    shp.field('ROT', 'N', 5)
    shp.field('NAV_STT', 'N', 2)
    for mmsi in list(Vs.keys()):
        for p in Vs[mmsi]:
            shp.point(p[LON],p[LAT])
            shp.record(p[MMSI],
                       p[TIMESTAMP],
                       time.strftime('%H:%M:%S %d-%m-%Y', time.gmtime(p[TIMESTAMP])),
                       p[LAT],
                       p[LON],
                       p[SOG],
                       p[COG],
                       p[HEADING],
                       p[ROT],
                       p[NAV_STT])
    shp.save(shp_fname)

#===============================================================================
#===============================================================================
def interpolate(t, track):
    """
    Interpolating the AIS message of vessel at a specific "t".
    INPUT:
        - t :
        - track     : AIS track, whose structure is
                     [LAT, LON, SOG, COG, HEADING, ROT, NAV_STT, TIMESTAMP, MMSI]
    OUTPUT:
        - [LAT, LON, SOG, COG, HEADING, ROT, NAV_STT, TIMESTAMP, MMSI]

    """

    before_p = np.nonzero(t >= track[:,TIMESTAMP])[0]
    after_p = np.nonzero(t < track[:,TIMESTAMP])[0]

    if (len(before_p) > 0) and (len(after_p) > 0):
        apos = after_p[0]
        bpos = before_p[-1]
        # Interpolation
        dt_full = float(track[apos,TIMESTAMP] - track[bpos,TIMESTAMP])
        if (abs(dt_full) > 2*3600):
            return None
        dt_interp = float(t - track[bpos,TIMESTAMP])
        try:
            az, _, dist = geod.inv(track[bpos,LON],
                                   track[bpos,LAT],
                                   track[apos,LON],
                                   track[apos,LAT])
            dist_interp = dist*(dt_interp/dt_full)
            lon_interp, lat_interp, _ = geod.fwd(track[bpos,LON], track[bpos,LAT],
                                               az, dist_interp)
            speed_interp = (track[apos,SOG] - track[bpos,SOG])*(dt_interp/dt_full) + track[bpos,SOG]
            course_interp = (track[apos,COG] - track[bpos,COG] )*(dt_interp/dt_full) + track[bpos,COG]
            heading_interp = (track[apos,HEADING] - track[bpos,HEADING])*(dt_interp/dt_full) + track[bpos,HEADING]
            #rot_interp = (track[apos,ROT] - track[bpos,ROT])*(dt_interp/dt_full) + track[bpos,ROT]
            if dt_interp > (dt_full/2):
                nav_interp = track[apos,NAV_STT]
            else:
                nav_interp = track[bpos,NAV_STT]
        except:
            return None
        return np.array([lat_interp, lon_interp,
                         speed_interp, course_interp,
                         heading_interp,
                         nav_interp,t,
                         track[0,MMSI]])
    else:
        return None

#===============================================================================
#===============================================================================
def remove_gaussian_outlier(v_data,quantile=1.64):
    """
    Remove outliers
    INPUT:
        v_data      : a 1-D array
        quantile    :
    OUTPUT:
        v_filtered  : filtered array
    """
    d_mean = np.mean(v_data)
    d_std = np.std(v_data)
    idx_normal = np.where(np.abs(v_data-d_mean)<=quantile*d_std)[0] #90%
    return v_data[idx_normal]

#===============================================================================
#===============================================================================
def gaussian_filter_with_nan(U,sigma):
    """
    Apply Gaussian filter when the data contain NaN
    INPUT:
        U           : a 2-D array (matrix)
        sigma       : std for the Gaussian kernel
    OUTPUT:
        Z           : filtered matrix
    """
    V=U.copy()
    V[np.isnan(U)]=0
    VV= ndimage.gaussian_filter(V,sigma=sigma)

    W=0*U.copy()+1
    W[np.isnan(U)]=0
    WW= ndimage.gaussian_filter(W,sigma=sigma)
    Z=VV/WW
    return(Z)

#===============================================================================
#===============================================================================
def show_logprob_map(m_map_logprob_mean, m_map_logprob_std, save_dir,
                     logprob_mean_min = -10.0, logprob_std_max = 5.0,
                     d_scale = 10, inter_method = "hanning",
                     fig_w = 960, fig_h = 960,
                    ):
    """
    Show the map of the mean and the std of the logprob in each cell.
    INPUT:
        m_map_logprob_mean   : a 2-D array (matrix)
        m_map_logprob_std    : a 2-D array (matrix)
        save_dir             : directory to save the images
    """
    # Truncate
    m_map_logprob_mean[m_map_logprob_mean<logprob_mean_min] = logprob_mean_min
    m_map_logprob_std[m_map_logprob_std>logprob_std_max] = logprob_std_max

    # Improve the resolution
    n_rows, n_cols = m_map_logprob_mean.shape
    m_mean = np.zeros((n_rows*d_scale,n_cols*d_scale))
    m_std = np.zeros((n_rows*d_scale,n_cols*d_scale))
    for i_row in range(m_map_logprob_mean.shape[0]):
        for i_col in range(m_map_logprob_mean.shape[1]):
            m_mean[d_scale*i_row:d_scale*(i_row+1),d_scale*i_col:d_scale*(i_col+1)] = m_map_logprob_mean[i_row,i_col]
            m_std[d_scale*i_row:d_scale*(i_row+1),d_scale*i_col:d_scale*(i_col+1)] = m_map_logprob_std[i_row,i_col]

    # Gaussian filter (with NaN)
    m_nan_idx = np.isnan(m_mean)
    m_mean = gaussian_filter_with_nan(m_mean, sigma=4.0)
    m_mean[m_nan_idx] = np.nan
    m_std = gaussian_filter_with_nan(m_std, sigma=4.0)
    m_nan_idx = np.isnan(m_std)
    m_std[m_nan_idx] = np.nan

    plt.figure(figsize=(fig_w/FIG_DPI, fig_h/FIG_DPI), dpi=FIG_DPI)
    # plt.subplot(1,2,1)
    im = plt.imshow(np.flipud(m_mean),interpolation=inter_method)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,"logprob_mean_map.png"))
    plt.close()

    plt.figure(figsize=(fig_w/FIG_DPI, fig_h/FIG_DPI), dpi=FIG_DPI)
    # plt.subplot(1,2,2)
    im = plt.imshow(np.flipud(m_std),interpolation=inter_method)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,"logprob_std_map.png"))
    plt.close()

#===============================================================================
#===============================================================================
def plot_abnormal_tracks(Vs_background,l_dict_anomaly,
                         filepath,
                         lat_min,lat_max,lon_min,lon_max,
                         onehot_lat_bins,onehot_lon_bins,
                         background_cmap = "Blues",
                         anomaly_cmap = "autumn",
                         l_coastline_poly = None,
                         fig_w = 960, fig_h = 960,
                         fig_dpi = 150,
                        ):
    plt.figure(figsize=(fig_w/FIG_DPI, fig_h/FIG_DPI), dpi=FIG_DPI)
    lat_range = lat_max - lat_min
    lon_range = lon_max - lon_min
    ## Plot background
    cmap = plt.cm.get_cmap(background_cmap)
    l_keys = list(Vs_background.keys())
    N = len(Vs_background)
    for d_i in range(N):
        key = l_keys[d_i]
        c = cmap(float(d_i)/(N-1))
        tmp = Vs_background[key]
        v_lat = tmp[:,0]*lat_range + lat_min
        v_lon = tmp[:,1]*lon_range + lon_min
        plt.plot(v_lon,v_lat,color=c,linewidth=0.8)
    plt.xlim([lon_min,lon_max])
    plt.ylim([lat_min,lat_max])
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.tight_layout()

    ## Coastlines
    try:
        for point in l_coastline_poly:
            poly = np.array(point)
            plt.plot(poly[:,0],poly[:,1],color="k",linewidth=0.8)
    except:
        pass

    ## Plot abnormal tracks
    cmap_anomaly = plt.cm.get_cmap(anomaly_cmap)
    N_anomaly = len(l_dict_anomaly)
    d_i = 0
    for D in l_dict_anomaly:
        try:
            c = cmap_anomaly(float(d_i)/(N_anomaly-1))
        except:
            c = 'r'
        d_i += 1
        tmp = D["seq"]
        m_log_weights_np = D["log_weights"]
        tmp = tmp[12:]
        v_lat = (tmp[:,0]/float(onehot_lat_bins))*lat_range + lat_min
        v_lon = ((tmp[:,1]-onehot_lat_bins)/float(onehot_lon_bins))*lon_range + lon_min
        plt.plot(v_lon,v_lat,color=c,linewidth=1.2)

    plt.savefig(filepath,dpi = fig_dpi)

In [ ]:
print("Removing anomalous message...")
error_count = 0
tick = time.time()
for k in  tqdm(list(voyages.keys())):
    track = voyages[k][:,[TIMESTAMP,LAT,LON,SOG]] # [Timestamp, Lat, Lon, Speed]
    try:
        o_report, o_calcul = detectOutlier(track, speed_max = 30)
        if o_report.all() or o_calcul.all():
            voyages.pop(k, None)
        else:
            voyages[k] = voyages[k][np.invert(o_report)]
            voyages[k] = voyages[k][np.invert(o_calcul)]
    except Exception as e:
        print(f"Error processing trajectory {k}: {e}")
        voyages.pop(k, None)
        error_count += 1
tok = time.time()
print("STEP 4: duration = ",(tok - tick)/60)

In [ ]:
print(len(voyages))

In [ ]:
error_count

In [ ]:
print('Sampling...')
Vs = dict()
count = 0
for k in tqdm(list(voyages.keys())):
    v = voyages[k]
    sampling_track = np.empty((0, 8))
    for t in range(int(v[0,TIMESTAMP]), int(v[-1,TIMESTAMP]), 300): # 5 min
        tmp = interpolate(t,v)
        if tmp is not None:
            sampling_track = np.vstack([sampling_track, tmp])
        else:
            sampling_track = None
            break
    if sampling_track is not None:
        Vs[count] = sampling_track
        count += 1

In [ ]:
count #flaw in interpolation function

In [ ]:
print('Re-Splitting...')
Data = dict()
count = 0
for k in tqdm(list(Vs.keys())):
    v = Vs[k]
    # Split AIS track into small tracks whose duration <= 1 day
    idx = np.arange(0, len(v), 12*DURATION_MAX)[1:]
    tmp = np.split(v,idx)
    for subtrack in tmp:
        # only use tracks whose duration >= 4 hours
        if len(subtrack) >= 12*4:
            Data[count] = subtrack
            count += 1
print(len(Data))

In [ ]:
print("Removing 'moored' or 'at anchor' voyages...")
for k in  tqdm(list(Data.keys())):
    d_L = float(len(Data[k]))

    if np.count_nonzero(Data[k][:,NAV_STT] == 1)/d_L > 0.7 \
    or np.count_nonzero(Data[k][:,NAV_STT] == 5)/d_L > 0.7:
        Data.pop(k,None)
        continue
    sog_max = np.max(Data[k][:,SOG])
    if sog_max < 1.0:
        Data.pop(k,None)
print("\n")
print(len(Data))

In [ ]:
print("Removing 'low speed' tracks...")
for k in tqdm(list(Data.keys())):
    d_L = float(len(Data[k]))
    if np.count_nonzero(Data[k][:,SOG] < 2)/d_L > 0.8:
        Data.pop(k,None)
print("\n")
print(len(Data))

In [ ]:
print('Normalisation...')
for k in tqdm(list(Data.keys())):
    v = Data[k]
    v[:,LAT] = (v[:,LAT] - LAT_MIN)/(LAT_MAX-LAT_MIN)
    v[:,LON] = (v[:,LON] - LON_MIN)/(LON_MAX-LON_MIN)
    v[:,SOG][v[:,SOG] > SPEED_MAX] = SPEED_MAX
    v[:,SOG] = v[:,SOG]/SPEED_MAX
    v[:,COG] = v[:,COG]/360.0

In [ ]:
output_filepath="/content/prodata/ct_2019/ct_2019_test.pkl"
print(output_filepath)


In [ ]:
print(os.path.dirname(output_filepath))

os.path.exists(os.path.dirname(output_filepath))


# In[27]:


if not os.path.exists(os.path.dirname(output_filepath)):
    os.makedirs(os.path.dirname(output_filepath))

In [ ]:
with open(output_filepath,"wb") as f:
    pickle.dump(Data,f)


In [ ]:
import pickle

# Path to the pickle file
pkl_file_path = "/content/prodata/ct_2019/ct_2019_test.pkl"

# Load the .pkl file
with open(pkl_file_path, "rb") as file:
    data = pickle.load(file)

# Print or inspect the data
print("Data loaded from pickle file:", data)

In [ ]:
for key in data.keys():
  print(len(data[key]))

In [ ]:
import pandas as pd
df1=pd.read_csv("/content/drive/MyDrive/data2/train.csv")
df1

In [ ]:
import numpy as np
len(np.unique(df1['MMSI']))